Задание 1.

Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

* Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
* Сделать нейронную сеть
* Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
* Проверить качество

Задание 2.

Выполнить практическую работу из лекционного ноутбука.

* Построить RNN-ячейку на основе полносвязных слоев
* Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [518]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
import numpy as np
import re

In [519]:
def f_cifer_cesar(phrase,num_K):
    
    abvgd = 'abcdefghijklmnopqrstuvwxyz' 
    if num_K>len(abvgd):
        num_K=len(abvgd)

    ab_ba=abvgd+abvgd[::-1]

    w_i_dict = {w: i for i, w in enumerate(abvgd)} 

    loc_lst=''
    for letter in phrase:
        if letter in abvgd:
            loc_lst+=ab_ba[w_i_dict[letter]+num_K]
        else:
            loc_lst+=letter
        
    return loc_lst

In [520]:
def f_clean_text(phrase):
    
    abvgd = 'abcdefghijklmnopqrstuvwxyz ' 

    w_i_dict = {w: i for i, w in enumerate(abvgd)} 

    loc_lst=''
    for letter in phrase.lower():
        if letter in abvgd:
            loc_lst+=abvgd[w_i_dict[letter]]
        else:
            loc_lst+=''
        
    return loc_lst

In [521]:
def f_text_to_num(phrase):
    
    abvgd = 'abcdefghijklmnopqrstuvwxyz ' 

    w_i_dict = {w: i for i, w in enumerate(abvgd)} 

    loc_lst=[]
    for letter in phrase.lower():
        if letter in abvgd:
            loc_lst.append(w_i_dict[letter])
        
    return torch.Tensor(loc_lst)

In [522]:
def f_num_to_text(lst):
    
    abvgd = 'abcdefghijklmnopqrstuvwxyz ' 
    lst=list(lst.numpy())
    i_w_dict = {i: w for i, w in enumerate(abvgd)} 

    loc_lst=''
    for num in lst:
        if num in range(len(abvgd)):
            loc_lst+=i_w_dict[num]
        
    return loc_lst

In [523]:
f_num_to_text(f_text_to_num('qwerty   uiop'))

'qwerty   uiop'

In [524]:
def f_cadr_text(phrase,lenght):

    loc_lst=phrase
    for i in range(lenght-len(phrase)):
        loc_lst+=' '
        
    return loc_lst

In [525]:
df = pd.read_csv('data.csv',usecols=['normalized_text'])

df=df.dropna().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10891 entries, 0 to 10890
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   normalized_text  10891 non-null  object
dtypes: object(1)
memory usage: 85.2+ KB


In [526]:
df['clean_text']=[f_clean_text(x) for x in df['normalized_text']]


In [527]:
lst_max=[len(x) for x in df['clean_text']]

lenght_max=lst_max[np.argmax(lst_max)]

lenght_max,'\n',df['clean_text'][np.argmax(lst_max)]

(392,
 '\n',
 'elegy for geezer rock postcard image thing to see to think of springfield is to think of thee what thoughts bepass ahind thy mien why sky art blue why trees art green and what pray tell did thine eyes see perchance old friend they gazed at me brought low by natures oafish hand thou crushed our reviewing stand and twixt thy stones glimpsed i the truth all things must pass  thy face my youth')

In [528]:
len(f_cadr_text(df['clean_text'][0],lenght_max))

392

In [529]:
df['cadr_text']=[f_cadr_text(x,lenght_max) for x in df['clean_text']]

In [530]:
K_cifer=25

In [531]:
df['cifer_text']=[f_cifer_cesar(x,K_cifer) for x in df['cadr_text']]
df.head()

,normalized_text,clean_text,cadr_text,cifer_text
0,maggie look whats that,maggie look whats that,maggie look whats that ...,ozuusw pmmq etzhi htzh ...
1,lee-mur lee-mur,leemur leemur,leemur leemur ...,pwwogj pwwogj ...
2,zee-boo zee-boo,zeeboo zeeboo,zeeboo zeeboo ...,bwwzmm bwwzmm ...
3,im trying to teach maggie that nature doesnt e...,im trying to teach maggie that nature doesnt e...,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...
4,its like an ox only it has a hump and a dewlap...,its like an ox only it has a hump and a dewlap...,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...


In [532]:
df=df[['cadr_text','cifer_text']]
df.head()

,cadr_text,cifer_text
0,maggie look whats that ...,ozuusw pmmq etzhi htzh ...
1,leemur leemur ...,pwwogj pwwogj ...
2,zeeboo zeeboo ...,bwwzmm bwwzmm ...
3,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...
4,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...


In [533]:
len(df['cadr_text'][0])

392

In [534]:
df['cifer_vec']=[f_text_to_num(x) for x in df['cifer_text']]
df['cadr_vec']=[f_text_to_num(x) for x in df['cadr_text']]



In [535]:
len(df['cifer_vec'][0])

392

In [536]:
df.head()

,cadr_text,cifer_text,cifer_vec,cadr_vec
0,maggie look whats that ...,ozuusw pmmq etzhi htzh ...,"[tensor(14.), tensor(25.), tensor(20.), tensor...","[tensor(12.), tensor(0.), tensor(6.), tensor(6..."
1,leemur leemur ...,pwwogj pwwogj ...,"[tensor(15.), tensor(22.), tensor(22.), tensor...","[tensor(11.), tensor(4.), tensor(4.), tensor(1..."
2,zeeboo zeeboo ...,bwwzmm bwwzmm ...,"[tensor(1.), tensor(22.), tensor(22.), tensor(...","[tensor(25.), tensor(4.), tensor(4.), tensor(1..."
3,im trying to teach maggie that nature doesnt e...,so hjcsnu hm hwzyt ozuusw htzh nzhgjw xmwinh w...,"[tensor(18.), tensor(14.), tensor(26.), tensor...","[tensor(8.), tensor(12.), tensor(26.), tensor(..."
4,its like an ox only it has a hump and a dewlap...,shi psqw zn md mnpc sh tzi z tgol znx z xwepzl...,"[tensor(18.), tensor(7.), tensor(8.), tensor(2...","[tensor(8.), tensor(19.), tensor(18.), tensor(..."


In [537]:
from sklearn.model_selection import  train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['cifer_vec'], df['cadr_vec'], test_size=0.2, random_state=42)

In [538]:
X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)

In [539]:
X_train=pd.Series([torch.Tensor(x).reshape(1,1,392) for x in X_train])
X_test= pd.Series([torch.Tensor(x).reshape(1,1,392) for x in  X_test])
y_train=pd.Series([torch.Tensor(x).reshape(1,1,392) for x in y_train])
y_test= pd.Series([torch.Tensor(x).reshape(1,1,392) for x in  y_test])

In [540]:
X_test.head()

0    [[[tensor(4.), tensor(22.), tensor(15.), tenso...
1    [[[tensor(13.), tensor(12.), tensor(26.), tens...
2    [[[tensor(7.), tensor(19.), tensor(25.), tenso...
3    [[[tensor(25.), tensor(13.), tensor(2.), tenso...
4    [[[tensor(7.), tensor(19.), tensor(25.), tenso...
dtype: object

In [541]:
len(X_test)

2179

In [542]:
for i in range(0, 5, 4):
    print(i)

0
4


In [543]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    for i in range(0, num_examples, batch_size):
        locX_tens=features[i]
        locY_tens=labels[i]

        if (i+batch_size)>num_examples:
            continue
        
        j = indices[i+1: i + batch_size]#min(i + batch_size, num_examples)]
        for k in j:
            locX_tens=torch.cat((locX_tens,features[k]))
            locY_tens=torch.cat((locY_tens,features[k]))

        yield locX_tens,locY_tens #features[j], labels[j]

In [544]:
x=torch.Tensor(1,1,392)
x.shape

torch.Size([1, 1, 392])

In [ ]:
batch_size = 256
for X, y in data_iter(batch_size, X_train, y_train):
    print(X.shape,'\n', y.shape)
    #break

In [558]:
'''model = torch.nn.Conv2d(in_channels=1,
                         out_channels=1,
                         kernel_size=(1,1))'''

'''model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(392, 2048),
    torch.nn.BatchNorm1d(2048),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(2048, 1024),
    torch.nn.BatchNorm1d(1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(512, 392)
)'''

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(392, 512),
    
    torch.nn.ReLU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(512, 392))#torch.nn.Flatten(),torch.nn.BatchNorm1d(512),


In [568]:
'''optimizer = torch.optim.SGD(conv2d.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()'''
loss = torch.nn.MSELoss(reduction='sum')
trainer = torch.optim.Adam(model.parameters(), lr=.001)

In [560]:
for X, y in data_iter(1, X_train, y_train):
            trainer.zero_grad()
            y_pred = model(X)
            print(X.shape,y.shape)
            break
            l = loss(y_pred, y)

torch.Size([1, 1, 392]) torch.Size([1, 1, 392])


In [566]:
'''m = torch.nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)

input.shape'''
num_epochs = 10
batch_size=1

def train_model():

    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for X, y in data_iter(batch_size, X_train, y_train):
            trainer.zero_grad()
            y_pred = model(X).squeeze()
            l = loss(y_pred, y.squeeze())
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred == y.squeeze()).sum().item()#(y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in data_iter(batch_size, X_test, y_test):
            y_pred = model(X).squeeze()
            l = loss(y_pred, y.squeeze())
            test_loss += l.item()
            test_acc += (y_pred == y.squeeze()).sum().item()#(y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
            
        print("epoch: {}, taked_time: {:.3f}, train_loss: {:.3f}, train_acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [562]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [569]:
num_epochs = 100
batch_size=1
train_model()

epoch: 0, taked_time: 80.471, train_loss: 6356.902, train_acc: 28.740, test_loss: 6281.906, test_acc: 4.995
epoch: 1, taked_time: 116.150, train_loss: 6340.898, train_acc: 5.619, test_loss: 6279.336, test_acc: 4.995
epoch: 2, taked_time: 102.616, train_loss: 6339.362, train_acc: 5.619, test_loss: 6279.356, test_acc: 4.995
epoch: 3, taked_time: 104.316, train_loss: 6339.237, train_acc: 5.619, test_loss: 6279.466, test_acc: 4.995
epoch: 4, taked_time: 103.901, train_loss: 6339.246, train_acc: 5.619, test_loss: 6279.515, test_acc: 4.995
epoch: 5, taked_time: 108.515, train_loss: 6339.257, train_acc: 5.619, test_loss: 6279.532, test_acc: 4.995
epoch: 6, taked_time: 117.579, train_loss: 6339.262, train_acc: 5.619, test_loss: 6279.538, test_acc: 4.995


KeyboardInterrupt: 

In [5]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:3]

['maggie look whats that', 'lee-mur lee-mur', 'zee-boo zee-boo']

In [ ]:
text = [[c for c in ph] for ph in phrases[:3] if type(ph) is str]
text

In [8]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [ ]:
INDEX_TO_CHAR

In [10]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])